In [1]:
import os

import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
def uploadDF(dataframe, eng, table_name):
    with eng.connect() as con:
        try:
            dataframe.to_sql(table_name, con, index=False)
        except Exception as e:
            con.rollback()
            print(f"[ERROR] Upload failed >>> {e}")
        else:
            con.commit()

# 1. Path definition

In [3]:
ROOT = "data/"

# Pazienti
path_pazienti = os.path.join(ROOT, "pazienti", "Pazienti_Dataset.xlsx")

# Cure
path_cure = os.path.join(ROOT, "cure", "Cure_Dataset.xlsx")

# Personale
path_personale = os.path.join(ROOT, "personale", "Personale_Dataset.xlsx")

# Reparti
path_reparti = os.path.join(ROOT, "reparti", "Reparto_Dataset.xlsx")

# Ricoveri
path_ricoveri = os.path.join(ROOT, "ricoveri", "Ricovero_Dateset.xlsx")

# Visualizzazioni
path_visualizzazioni = os.path.join(ROOT, "visualizzazioni", "Visualizzazioni_Dataset.xlsx")

# Parametri vitali
path_subject00 = os.path.join(ROOT, "parametriVitali", "subject_00.xlsx")
path_subject01 = os.path.join(ROOT, "parametriVitali", "subject_01.xlsx")
path_subject02 = os.path.join(ROOT, "parametriVitali", "subject_02.xlsx")
path_subject03 = os.path.join(ROOT, "parametriVitali", "subject_03.xlsx")
path_subject04 = os.path.join(ROOT, "parametriVitali", "subject_04.xlsx")
path_subject05 = os.path.join(ROOT, "parametriVitali", "subject_05.xlsx")
path_subject06 = os.path.join(ROOT, "parametriVitali", "subject_06.xlsx")
path_subject08 = os.path.join(ROOT, "parametriVitali", "subject_08.xlsx")
path_subject09 = os.path.join(ROOT, "parametriVitali", "subject_09.xlsx")
path_subject10 = os.path.join(ROOT, "parametriVitali", "subject_10.xlsx")

# 2. Loading DataFrame

In [4]:
df_pazienti = pd.read_excel(path_pazienti, header=0)
df_cure = pd.read_excel(path_cure, header=0)
df_personale = pd.read_excel(path_personale, header=0)
df_reparti = pd.read_excel(path_reparti, header=0)
df_ricoveri = pd.read_excel(path_ricoveri, header=0)
df_visualizzazioni = pd.read_excel(path_visualizzazioni, header=0)

In [5]:
df_00 = pd.read_excel(path_subject00, header=0)
df_01 = pd.read_excel(path_subject01, header=0)
df_02 = pd.read_excel(path_subject02, header=0)
df_03 = pd.read_excel(path_subject03, header=0)
df_04 = pd.read_excel(path_subject04, header=0)
df_05 = pd.read_excel(path_subject05, header=0)
df_06 = pd.read_excel(path_subject06, header=0)
df_08 = pd.read_excel(path_subject08, header=0)
df_09 = pd.read_excel(path_subject09, header=0)
df_10 = pd.read_excel(path_subject10, header=0)

df_parametri = pd.concat([df_00, df_01, df_02, df_03, df_04, df_05, df_06, df_08, df_09, df_10])

del df_00, df_01, df_02, df_03, df_04, df_05, df_06, df_08, df_09, df_10

# 3. Loading data to DB

In [6]:
drivername = "mysqlconnector"
user = "root"
pw = "ciao"
server = "127.0.0.1"
port = "3306"

db_name = "fabecare_db"

## 3.1 Create DB

In [7]:
first_conn = f"mysql+{drivername}://{user}:{pw}@{server}:{port}"
engine = create_engine(first_conn)

with engine.connect() as conn:
    try:
        conn.execute(text(f"CREATE DATABASE {db_name}"))
    except:
        conn.rollback()
    else:
        conn.commit()

In [8]:
conn_eng = f"mysql+{drivername}://{user}:{pw}@{server}:{port}/{db_name}"
engine = create_engine(conn_eng)

## 3.2 Loading tables

In [9]:
uploadDF(df_reparti, engine, "reparti")
uploadDF(df_pazienti, engine, "pazienti")
uploadDF(df_personale, engine, "personale")
uploadDF(df_ricoveri, engine, "ricoveri")
uploadDF(df_cure, engine, "cure")

In [10]:
uploadDF(df_visualizzazioni, engine, "visualizzazioni")

In [11]:
uploadDF(df_parametri, engine, "parametri_vitali")

## 3.3 Defining PRIMARY KEYs

In [12]:
with engine.connect() as conn:
    try:
        conn.execute(text("ALTER TABLE reparti MODIFY IDR VARCHAR(50)"))
        conn.execute(text("ALTER TABLE reparti MODIFY DENOMINAZIONE VARCHAR(500)"))
        conn.execute(text("ALTER TABLE reparti ADD PRIMARY KEY (IDR)"))
        
        conn.execute(text("ALTER TABLE pazienti MODIFY IDP VARCHAR(50)"))
        conn.execute(text("ALTER TABLE pazienti ADD PRIMARY KEY (IDP)"))
        
        conn.execute(text("ALTER TABLE parametri_vitali MODIFY ID VARCHAR(50)"))
        conn.execute(text("ALTER TABLE parametri_vitali MODIFY TIPO VARCHAR(50)"))
        conn.execute(text("ALTER TABLE parametri_vitali ADD PRIMARY KEY (ID, TIMESTAMP, TIPO)"))
        
        conn.execute(text("ALTER TABLE personale MODIFY ID_PERS VARCHAR(50)"))
        conn.execute(text("ALTER TABLE personale MODIFY REPARTO VARCHAR(500)"))
        conn.execute(text("ALTER TABLE personale MODIFY ID_REPARTO VARCHAR(50)"))
        conn.execute(text("ALTER TABLE personale ADD PRIMARY KEY (ID_PERS)"))
        
        conn.execute(text("ALTER TABLE ricoveri MODIFY ID_PAZIENTI VARCHAR(50)"))
        conn.execute(text("ALTER TABLE ricoveri MODIFY ID_REPARTO VARCHAR(50)"))
        conn.execute(text("ALTER TABLE ricoveri ADD PRIMARY KEY (ID_PAZIENTI)"))
        
        conn.execute(text("ALTER TABLE cure MODIFY ID_PAZIENTE VARCHAR(50)"))
        conn.execute(text("ALTER TABLE cure MODIFY ID_PERSONALE VARCHAR(50)"))
        conn.execute(text("ALTER TABLE cure ADD PRIMARY KEY (ID_PAZIENTE, ID_PERSONALE)"))
        
        conn.execute(text("ALTER TABLE visualizzazioni MODIFY ID_PAZIENTE VARCHAR(50)"))
        conn.execute(text("ALTER TABLE visualizzazioni MODIFY ID_PERSONALE VARCHAR(50)"))
        conn.execute(text("ALTER TABLE visualizzazioni MODIFY TIPOLOGIA VARCHAR(50)"))
        conn.execute(text("ALTER TABLE visualizzazioni ADD PRIMARY KEY (ID_PAZIENTE, ID_PERSONALE, TIPOLOGIA, TEMPO_ACQUISIZIONE)"))
        
    except Exception as e:
        conn.rollback()
        print(f"[ERROR] {e}")
    else:
        conn.commit()

## 3.4 Defining references

In [13]:
with engine.connect() as conn:
    try:
        conn.execute(text("ALTER TABLE parametri_vitali ADD FOREIGN KEY parametri_vitali(ID) REFERENCES pazienti(IDP)"))
        
        conn.execute(text("ALTER TABLE personale ADD FOREIGN KEY personale(ID_REPARTO) REFERENCES reparti(IDR)"))
        
        conn.execute(text("ALTER TABLE ricoveri ADD FOREIGN KEY ricoveri(ID_REPARTO) REFERENCES reparti(IDR)"))
        conn.execute(text("ALTER TABLE ricoveri ADD FOREIGN KEY ricoveri(ID_PAZIENTI) REFERENCES pazienti(IDP)"))
        
        conn.execute(text("ALTER TABLE cure ADD FOREIGN KEY cure(ID_PAZIENTE) REFERENCES pazienti(IDP)"))
        conn.execute(text("ALTER TABLE cure ADD FOREIGN KEY cure(ID_PERSONALE) REFERENCES personale(ID_PERS)"))
        
        conn.execute(text("ALTER TABLE visualizzazioni ADD FOREIGN KEY visualizzazioni(ID_PAZIENTE) REFERENCES pazienti(IDP)"))
        conn.execute(text("ALTER TABLE visualizzazioni ADD FOREIGN KEY visualizzazioni(ID_PERSONALE) REFERENCES personale(ID_PERS)"))
    
    except Exception as e:
        print(f"[ERROR] {e}")
        conn.rollback()
        
    else:
        conn.commit()